# 选课系统

tools
1. 过滤课程tool：过滤选修，必修
2. 插入用户名称和对应课程tool
3. 删除用户名称和对应课程tool

agent行为
1. agent 根据需要选择课程，且将和用户描述比较相似的排在前面
2. agent 做好选课和删除课程名称匹配，用户不必说出课程全名
3. agent 

## 数据存储

存储课程

In [2]:
course_dict = {
    "required":[
        "software energening",
        "machine learning",
        "math",
        "physical"
    ],
    "elective":[
        "tennis",
        "table tennis",
        "football"
    ]
}

In [5]:
from langchain.agents import tool


# 过滤课程tool，llm输入课程类型，过滤对应类型结果
@tool
def filter_course(course_type:str)->str:
    """
        return course list, use this for filtering courses,
        there two kinds of courses: required , elective
        the input is string type and the content should be required or elective
        the function will return the course list
    """
    return course_dict[course_type]

## 初始化llm

In [3]:
# 初始化llm
from langchain.chat_models import ChatOpenAI
import os
llm_model = "Qwen2.5-14B"
os.environ["OPENAI_API_KEY"] = "None"
os.environ["OPENAI_API_BASE"] = "http://10.58.0.2:8000/v1"
# llm_completion = OpenAI(model_name="Qwen2.5-14B") 
# llm_chat = OpenAIChat(model_name="Qwen2.5-14B")
llm = ChatOpenAI(temperature=0, model=llm_model)


/var/folders/42/s_p9v7315ks8j7qscyd35d5c0000gn/T/ipykernel_38095/865272866.py:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model=llm_model)


In [13]:
import langchain
langchain.debug = True
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType

# tools = load_tools(['llm-math'],llm=llm)
tools = load_tools(["llm-math"], llm=llm) 

agent= initialize_agent(
    tools+[filter_course], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

## 用户查询课程

可查询选修或者必修

In [17]:
user_course_type_question = "有哪些是必修课?"

In [18]:
agent.run(
    f"""
        filter the course list,there two kinds of courses
        first is required,second is elective 
        should baded on users input to filter the correct courses

        <<< COURSE DICT >>>

        {course_dict}

        <<< USER INPUT >>>

        {user_course_type_question}
    """
)

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "\n        filter the course list,there two kinds of courses\n        first is required,second is elective \n        should baded on users input to filter the correct courses\n\n        <<< COURSE DICT >>>\n\n        {'required': ['software energening', 'machine learning', 'math', 'physical'], 'elective': ['tennis', 'table tennis', 'football']}\n\n        <<< USER INPUT >>>\n\n        有哪些是必修课?\n    "
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "\n        filter the course list,there two kinds of courses\n        first is required,second is elective \n        should baded on users input to filter the correct courses\n\n        <<< COURSE DICT >>>\n\n        {'required': ['software energening', 'machine learning', 'math', 'physical'], 'elective': ['tennis', 'table tennis', 'football']}\n\n        <<< USER INPUT >>>\n\n        有哪些是必修课?\n    ",
  "agent_scr

'必修课有：软件工程、机器学习、数学、物理。'